# This notebook is investigating TB and MOD data for the Hunza met station locations




## Load in all the modules needed

In [1]:
%pylab notebook
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import shapely.geometry as sgeom
import matplotlib.pyplot as plt
from netCDF4 import Dataset, num2date
import numpy as np
import pandas as pd
import rasterio
from cetbtools.ease2conv import Ease2Transform
import time

Populating the interactive namespace from numpy and matplotlib


In [2]:
# navigate to where scripts are saved
%cd /projects/brodzik/ipynb_melt_onset/scripts/
%ls

/projects/brodzik/ipynb_melt_onset/scripts
CETB_algorithms.py  CETB_analysis.py~       __pycache__/
CETB_analysis.py    CETB_read_functions.py


In [3]:
# load the custom functions
from CETB_read_functions import read_Tb
from CETB_read_functions import coords
from CETB_read_functions import calc_DAV
from CETB_read_functions import find_UIB_cube_offset
from CETB_read_functions import grid_locations_of_UIB
from CETB_algorithms import DAV_MOD
from CETB_analysis import MOD_array
from CETB_analysis import MOD_array_year

In [4]:
# Set pixel lookups by met station
# ref: /Users/brodzik/Desktop/GIS_data/Pakistan/Hunza/wapda_met_stations_CETBlocs.v4.xlsx
Khunjerab = {"name": "Khunjerab",
             "lat": 36.8411,
             "lon": 75.4192,
             "row3km": 220,
             "col3km": 41,
             "row25km": 27,
             "col25km": 5
           }

In [5]:
station = Khunjerab
station

{'name': 'Khunjerab',
 'lat': 36.8411,
 'lon': 75.4192,
 'row3km': 220,
 'col3km': 41,
 'row25km': 27,
 'col25km': 5}

## Read TB data

In [6]:
#Specify region, satellite, sensor, channel, and image reconstruction algorithm of interest in file name
# this notebook will read in 2 CETB datasets so that channels/algorithms/sensors can be compared
region='UIB'  #make this the same syntax as cubefilenames and sub-directory
platform='AQUA'   #'AQUA' for AMSRE, 'F13','F14','F15'... for SSMI
sensor='AMSRE'  #'AMSRE', 'SSMI', etc.
channel='36V'  #'36V','36H', '18V','18H', etc. '19V','19H' and '37V','37H' for SSMI)
version='v1.3'
proj='N'

if sensor=='SSMI':
    provider='CSU' 
elif sensor=='AMSRE':
    provider='RSS'

cubeDir = '/work/PMESDR/CETB_v1.3/%s_%s/%s/cubes_%s/' % (platform, sensor, proj, region)    

# prefix filepath
prefix_GRD = 'CETB.cubefile.%s.%s_%s-%s-GRD-%s-%s' % (region, platform, sensor, channel, provider, version)
prefix_SIR = 'CETB.cubefile.%s.%s_%s-%s-SIR-%s-%s' % (region, platform, sensor, channel, provider, version)

# years for each sensor
# F13, May 95 - Nov 09
if platform=='F13':
    # F13, May 95 - Nov 09
    Years = [2002,2003,2004,2005,2006,2007,2008,2009]
elif platform=='F14':
    # F14, May 97 - Aug 08
    Years=[2002,2003,2004,2005,2006,2007,2008]
elif platform=='F15':
    # F15, Feb 00 - Jun 17
    Years=[2002,2003,2004,2005,2006,2007,2008,2009,2010,2011]
elif platform=='AQUA':
    # AQUA AMSR-E: Jun 02 - Oct 11
    Years=[2003,2004,2005,2006,2007,2008,2009,2010,2011]


In [7]:
# Set up cartopy CRSs for E2N and 
# for UIB cubes, I want to rotate 90 degrees clockwise
geod = ccrs.Geodetic()
e2n = ccrs.LambertAzimuthalEqualArea(central_latitude=90.0)
e2nRotate = ccrs.LambertAzimuthalEqualArea(central_latitude=90.0, central_longitude=90.0)

# Sanity check
lon, lat = geod.transform_point(
    x = -9000000.,
    y = 9000000.,
    src_crs=e2n)
print("E2N UL corner lon=%.4f, lat=%.4f" % (lon, lat))

E2N UL corner lon=-135.0000, lat=-84.6340


# Read the UIB shapefile to get the bounds of the cube area

Read the basin outline shapefiles
Use shapely to read the .shp files. This works for lonlat shapefiles, it doesn't seem to work for projected ones, there must be an option that I'm just missing.

Use the cartopy CRS to project the shapefile to E2N

In [8]:
hunzaBasinfile = '/work/charis/ti_model/basins/basin_outlines/IN_Hunza_at_DainyorBridge.shp'
hunzaReader = shpreader.Reader(hunzaBasinfile)
hunzaRecord = next(hunzaReader.records())
#hunzaRecord, hunzaRecord.attributes, hunzaRecord.bounds, hunzaRecord.geometry

## Rotate/project the basin outline shapefiles

In [9]:
e2nRotateHunzaBasin = e2nRotate.project_geometry(hunzaRecord.geometry)

In [10]:
bounds = hunzaRecord.bounds
Site = "Hunza"
#bounds = UIBRecord.bounds
#Site = "UIB"
bounds

(74.02507731119763, 35.92307128906174, 75.77779744466116, 37.09783732096277)

In [11]:
station

{'name': 'Khunjerab',
 'lat': 36.8411,
 'lon': 75.4192,
 'row3km': 220,
 'col3km': 41,
 'row25km': 27,
 'col25km': 5}

<code>
forward_grid:
enter lat lon: 36.8411 75.4192
col,row = 580.679566 417.033822    status = 1
lat,lon = 36.841100 75.419200    status = 1
enter lat lon: 
    </code>

In [15]:
# get the GRD pixel IDs for station location
rows_cols_GRD=coords(cubeDir, prefix_GRD, station['lat'], station['lat'], station['lon'], station['lon'])
rows_cols_GRD

(26, 27, 6, 7)

In [16]:
# get the GRD pixel IDs for station location
rows_cols_env=coords(cubeDir, prefix_SIR, station['lat'], station['lat'], station['lon'], station['lon'])
rows_cols_env

(210, 211, 54, 55)

In [ ]:
27*8

In [31]:
# load in SIR TB data
data_SIR=read_Tb(cubeDir, prefix_SIR, Years)
CETB_SIR=data_SIR['TB']   # 3-D Tb time-series array of TB
cal_date=data_SIR['cal_date']    # 1-D array of dates, these will get passed to later functions
cal_year=data_SIR['cal_year']    # 1-D array of years
cal_month=data_SIR['cal_month']   # 1-D array of months

# load GRD Tb data
data_GRD=read_Tb(cubeDir, prefix_GRD, Years)
CETB_GRD=data_GRD['TB']

No subset specified, fetching complete cube...
No subset specified, fetching complete cube...


In [20]:
# Compare Mitch's coords answer to the one I get with mapx

In [32]:
#mitch_lat = data_GRD['latitude'][rows_cols_GRD[0]][rows_cols_GRD[2]]
#mitch_lon = data_GRD['longitude'][rows_cols_GRD[0]][rows_cols_GRD[2]]
mitch_lat = data_SIR['latitude'][rows_cols_env[0]][rows_cols_env[2]]
mitch_lon = data_SIR['longitude'][rows_cols_env[0]][rows_cols_env[2]]
#rows_cols_GRD, mitch_lat, mitch_lon
rows_cols_env, mitch_lat, mitch_lon

((210, 211, 54, 55), 36.51889472705087, 75.81636978500872)

In [33]:
#my_lat = data_GRD['latitude'][station['row25km']][station['col25km']]
#my_lon = data_GRD['longitude'][station['row25km']][station['col25km']]
#station, my_lat, my_lon
my_lat = data_SIR['latitude'][station['row3km']][station['col3km']]
my_lon = data_SIR['longitude'][station['row3km']][station['col3km']]
station, my_lat, my_lon

({'name': 'Khunjerab',
  'lat': 36.8411,
  'lon': 75.4192,
  'row3km': 220,
  'col3km': 41,
  'row25km': 27,
  'col25km': 5},
 36.837347378374396,
 75.41274294152329)

In [34]:
my_lat - station['lat'], mitch_lat - station['lat']

(-0.0037526216256011935, -0.3222052729491267)

In [35]:
my_lon - station['lon'], mitch_lon - station['lon']

(-0.006457058476712518, 0.3971697850087139)

In [ ]:
# calculate DAV for the Tb data that was imported
DAV_SIR=calc_DAV(CETB_SIR)
DAV_GRD=calc_DAV(CETB_GRD)

In [ ]:
# Given a cube's (or subset's) x, y values
# calculate the extent of the edges 
# as 1/2 pixel larger than centers of corner pixels
def get_extent_xy(x, y):
    # Assumes pixels with regular spacing
    scale_x = x[1] - x[0]
    scale_y = y[1] - y[0]
    print("scales: ", scale_x, scale_y)
    
    extent = [x[0] - (scale_x/2.),
              x[-1] + (scale_x/2.),
              y[-1] + (scale_y/2.),
              y[0] - (scale_y/2.)]
              
    return extent

## Check extents of full cube

In [ ]:
data_GRD['x'].shape, data_GRD['y'].shape

In [ ]:
get_extent_xy(data_SIR['x'], data_SIR['y'])

In [ ]:
get_extent_xy(data_GRD['x'], data_GRD['y'])

## Check extents of requested subsets

In [ ]:
get_extent_xy(data_SIR['x'][rows_cols_env[2]:rows_cols_env[3]],
              data_SIR['y'][rows_cols_env[0]:rows_cols_env[1]])

In [ ]:
get_extent_xy(data_GRD['x'][rows_cols_GRD[2]:rows_cols_GRD[3]],
              data_GRD['y'][rows_cols_GRD[0]:rows_cols_GRD[1]])

## Read DEMs at 3.125 and 25 km for displays

In [ ]:
DEM25kmFile = '/work/charis/ti_model/SRTMGL3_version2_EASE2/EASE2_N25km.CHARIS_DEM_v2_IN.UIB_Khan_clip.mode.tif'
with rasterio.open(DEM25kmFile) as src:
    dem25km = np.squeeze(src.read())
DEM3kmFile = '/work/charis/ti_model/SRTMGL3_version2_EASE2/EASE2_N3.125km.CHARIS_DEM_v2_IN.UIB_Khan_clip.mode.tif'
with rasterio.open(DEM3kmFile) as src:
    dem3km = np.squeeze(src.read())

## create arrays of MOD to be used for plotting

In [ ]:
# get array of average MODs for SIR pixels for all the years loaded
window=10   # window for MOD algorithm, '10' would be 5 days (2 measurements per day)
count=3    # number of Tb/DAV exceedances to trigger MOD
DAV_threshold=40
Tb_threshold=252

In [ ]:
Years

In [ ]:
# sir MOD array - MOD will be in day of year (DOY)
t = time.process_time()
MOD_DOY_array=MOD_array(cubeDir, prefix_SIR, CETB_SIR, DAV_SIR, 
                        rows_cols_env, cal_date, Years, window, count, DAV_threshold, Tb_threshold)
elapsed_time = time.process_time() - t
MOD_DOY_array, elapsed_time

In [ ]:
# MOD of the GRD pixel - avg all years
#window=10
#count=3
#DAV_threshold=18
#Tb_threshold=252
t = time.process_time()
MOD_DOY_array_GRD=MOD_array(cubeDir, prefix_GRD, CETB_GRD, DAV_GRD, 
                            rows_cols_GRD, cal_date, Years, window, count, DAV_threshold, Tb_threshold)
elapsed_time = time.process_time() - t
MOD_DOY_array_GRD, elapsed_time

In [ ]:
# Save these arrays to numpy zip
outfile = '/projects/brodzik/pmesdr_melt_onset/data/%s_%s_%s_%d-%d_MOD.DAV%03d.Tb%03d.npz' % (
    Site, platform, sensor, Years[0], Years[-1], DAV_threshold, Tb_threshold)
np.savez(outfile, 
         MOD_DOY_array=MOD_DOY_array, 
         MOD_DOY_array_GRD=MOD_DOY_array_GRD)
print("Avg SIR and GRD MODs saved to %s" % outfile)

In [ ]:
# For each individual year, do SIR and GRD MOD

# get array of MOD for each pixel SIR - one year of interest
#window=10
#count=3
#DAV_threshold=18
#Tb_threshold=252

# For each individual year, do SIR MOD
for year in Years:
    print("year = %d" % year)
    t = time.process_time()
    MOD_DOY_array_year=MOD_array_year(cubeDir, prefix_SIR, CETB_SIR, DAV_SIR, 
                                      rows_cols_env, cal_date, year, 
                                      window, count, DAV_threshold, Tb_threshold)
    elapsed_time = time.process_time() - t
    print("SIR elapsed time = %f" % elapsed_time)
    
    t = time.process_time()
    MOD_DOY_array_GRD_year=MOD_array_year(cubeDir, prefix_GRD, CETB_GRD, DAV_GRD, 
                                          rows_cols_GRD, cal_date, year, 
                                          window, count, DAV_threshold, Tb_threshold)
    elapsed_time = time.process_time() - t
    print("GRD elapsed time = %f" % elapsed_time)
    
    outfile = '/projects/brodzik/pmesdr_melt_onset/data/%s_%s_%s_%d_MOD.DAV%03d.Tb%03d.npz' % (
        Site, platform, sensor, year, DAV_threshold, Tb_threshold)
    np.savez(outfile, 
             MOD_DOY_array_year=MOD_DOY_array_year,
             MOD_DOY_array_GRD_year=MOD_DOY_array_GRD_year)
    print("MOD data saved to %s" % outfile)

## Load data produced earlier

In [ ]:
# This will need to be changed to read the particular year of interest
# Load data saved on previous runs
outfile = '/projects/brodzik/pmesdr_melt_onset/data/%s_%s_%s_%d-%d_MOD.DAV%03d.Tb%03d.npz' % (
    Site, platform, sensor, Years[0], Years[-1], DAV_threshold, Tb_threshold)
npzfile = np.load(outfile)
print("Loading avg data from %s" % outfile)
#npzfile.files
# should return 2 avg MOD variable names as a dict
MOD_DOY_array = npzfile['MOD_DOY_array']
MOD_DOY_array_GRD = npzfile['MOD_DOY_array_GRD']

In [ ]:
# Set the year to read
year = 2011
outfile = '/projects/brodzik/pmesdr_melt_onset/data/%s_%s_%s_%d_MOD.DAV%03d.Tb%03d.npz' % (
        Site, platform, sensor, year, DAV_threshold, Tb_threshold)
npzfile = np.load(outfile)
print("Loading year=%d data from %s" % (year, outfile))
MOD_DOY_array_year = npzfile['MOD_DOY_array_year']
MOD_DOY_array_GRD_year = npzfile['MOD_DOY_array_GRD_year']

# Convert the MOD information back to an image for display

In [ ]:
MOD_DOY = MOD_DOY_array.reshape((rows_cols_env[1] - rows_cols_env[0],
                                 rows_cols_env[3] - rows_cols_env[2]))
MOD_DOY_year = MOD_DOY_array_year.reshape((rows_cols_env[1] - rows_cols_env[0],
                                           rows_cols_env[3] - rows_cols_env[2]))
                                         
MOD_DOY_GRD = MOD_DOY_array_GRD.reshape((rows_cols_GRD[1] - rows_cols_GRD[0],
                                         rows_cols_GRD[3] - rows_cols_GRD[2]))
MOD_DOY_GRD_year = MOD_DOY_array_GRD_year.reshape((rows_cols_GRD[1] - rows_cols_GRD[0],
                                                   rows_cols_GRD[3] - rows_cols_GRD[2]))

In [ ]:
fig = plt.figure(figsize=(14,6))

# Figure out the extent to display and rotate the extent by 90 degrees
# This extent is for the current selected subset of the UIB cube
cube_extent=get_extent_xy(data_GRD['x'][rows_cols_GRD[2]:rows_cols_GRD[3]], 
                          data_GRD['y'][rows_cols_GRD[0]:rows_cols_GRD[1]])
cube_extentRotate=[cube_extent[2], cube_extent[3], -1.*cube_extent[1], -1. * cube_extent[0]]

# Placeholders for the various elements of each subplot
axes = []
images = []

numrows = 2
numcols = 3
numplots = numrows * numcols

labels = ["SRTMGL3 DEM (3.125 km)",
          "%s %s SIR Avg MOD (%4d-%4d)" % (platform, sensor, Years[0], Years[-1]),
          "%s %s SIR MOD (%s)" % (platform, sensor, year),
          "SRTMGL3 DEM (25 km)",
          "%s %s GRD Avg MOD (%4d-%4d)" % (platform, sensor, Years[0], Years[-1]),
          "%s %s GRD MOD (%s)" % (platform, sensor, year)]
color_labels = ["Elevation (m)",
                "DOY",
                "DOY",
                "Elevation (m)",
                "DOY",
                "DOY"]

# Set the projection and extent for each subplot
for i in np.arange(numplots):
    axes.append(fig.add_subplot(numrows, numcols, i+1, projection=e2nRotate))
    axes[i].set_extent(cube_extentRotate, crs=e2nRotate)

images.append(axes[0].imshow(np.rot90(
    dem3km[rows_cols_env[0]:rows_cols_env[1], rows_cols_env[2]:rows_cols_env[3]], -1),
                             extent=cube_extentRotate, 
                             transform=e2nRotate,
                             origin='upper', cmap='Greys_r', 
                             interpolation='None', vmin=np.min(150), vmax=np.max(8000), label=labels[0]))

images.append(axes[1].imshow(np.rot90(MOD_DOY, -1), 
                             extent=cube_extentRotate, 
                             transform=e2nRotate,
                             origin='upper', cmap='RdBu', 
                             interpolation='None', vmin=np.min(75), vmax=np.max(170), label=labels[1]))

images.append(axes[2].imshow(np.rot90(MOD_DOY_year, -1), 
              extent=cube_extentRotate, 
              transform=e2nRotate,
              origin='upper', cmap='RdBu', 
              interpolation='None', vmin=np.min(75), vmax=np.max(170), label=labels[2]))

images.append(axes[3].imshow(np.rot90(
    dem25km[rows_cols_GRD[0]:rows_cols_GRD[1], rows_cols_GRD[2]:rows_cols_GRD[3]], -1),
                             extent=cube_extentRotate, 
                             transform=e2nRotate,
                             origin='upper', cmap='Greys_r', 
                             interpolation='None', vmin=np.min(150), vmax=np.max(8000), label=labels[3]))

images.append(axes[4].imshow(np.rot90(MOD_DOY_GRD, -1), 
                           extent=cube_extentRotate, 
                           transform=e2nRotate,
                           origin='upper', cmap='RdBu', 
                           interpolation='None', vmin=np.min(75), vmax=np.max(170), label=labels[4]))

images.append(axes[5].imshow(np.rot90(MOD_DOY_GRD_year, -1), 
              extent=cube_extentRotate, 
              transform=e2nRotate,
              origin='upper', cmap='RdBu', 
              interpolation='None', vmin=np.min(75), vmax=np.max(170), label=labels[5]))

# Do common display stuff for each subplot
for i in np.arange(numplots):
    axes[i].axis('off')
    axes[i].gridlines(color='gray', linestyle='--')
    axes[i].coastlines()
    axes[i].add_geometries([e2nRotateUIBBasin], 
                      e2nRotate,
                      edgecolors='black', facecolor='none', lw=2)
    axes[i].add_geometries([e2nRotateHunzaBasin], e2nRotate,
                      edgecolors='red', facecolor='none', lw=2)
    #For separate colorbars:
    cbar = fig.colorbar(images[i], ax=axes[i])
    cbar.ax.set_ylabel(color_labels[i])
    axes[i].set_title(labels[i])

plt.tight_layout()
plt.show()
pngfile = "/projects/brodzik/pmesdr_melt_onset/data/%s_%s_%s_%d-%d_w%d.MOD.DAV%03d.Tb%03d.png" % (
    Site, platform, sensor, Years[0], Years[-1], year, DAV_threshold, Tb_threshold)
fig.savefig(pngfile, dpi=300)
print("Saved plot to %s" % pngfile)